# Email Spam Detector

## Import library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import tensorflow.keras
import datetime
from tensorflow.keras import backend as K
import tensorflow.keras.optimizers as Optimizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, load_model
# import tensorflow_addons as tfa

from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from sklearn.metrics import confusion_matrix as CM
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
import matplotlib.pyplot as plot
import seaborn as sn
import tensorflow as tf

## Import dataset

Business Understanding:

What is mail spam ?

Spam emails are illegal or unsolicited messages sent through to a large number of users. 

Their primary goal is to lure the user into clicking a malicious link or downloading an attachment that is harmful to the user's machine.

Why classifycation mail spam?

to protect user from malicious email

reduce user distractions when receiving unsolicited emails

Goal: 

build a usable spam classification model

The classification is based on the f1 metric, which is more than 0.9

In [2]:
email = pd.read_csv('https://raw.githubusercontent.com/VuongDS/305/main/emails_1.csv')

Shape of dataset is (5728, 2).
It's conclude two column:
- 'text' contains the content of the email
- 'spam' is label of dataset

In [3]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

## Cleaning the text

In [4]:
from gensim.utils import simple_preprocess

In [5]:
email.iloc[:, 0] = email.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))

In [6]:
email_spam = email[email['spam'] == 1] 
email_not_spam = email[email['spam'] == 0] 

In [7]:
# Store word in a list:
text_contain_spam = []
for text in email_spam.text:
  text_contain_spam.append(text)


text_contain_not_spam = []
for text in email_not_spam.text:
  text_contain_not_spam.append(text)

In [8]:
# Conver list to a text file
text_contain_spam = ''.join(text_contain_spam)
text_contain_not_spam = ''.join(text_contain_not_spam)

## error analysis

In [9]:
email.head()

,text,spam
0,subject naturally irresistible your corporate ...,1
1,subject the stock trading gunslinger fanny is ...,1
2,subject unbelievable new homes made easy im wa...,1
3,subject color printing special request additio...,1
4,subject do not have money get software cds fro...,1


In [10]:
email_ea = email.copy()

In [11]:
email_ea.at[2239, 'spam'] = 0
email_ea.at[1180, 'spam'] = 0
email_ea.at[499, 'spam'] = 0
email_ea.at[737, 'spam'] = 0
email_ea.at[882, 'spam'] = 0
email_ea.at[401, 'spam'] = 0
email_ea.at[1086, 'spam'] = 0

# email_ea.at[1181, 'spam'] = 0
# email_ea.at[3523, 'spam'] = 1
# email_ea.at[3728, 'spam'] = 1
# email_ea.at[3513, 'spam'] = 1

## Augmentation data

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train_au, X_test_au, y_train_au, y_test_au = train_test_split(email_ea['text'], email_ea['spam'], test_size = 0.20, random_state = 42)
X_train_au = pd.DataFrame(X_train_au)
X_test_au = pd.DataFrame(X_test_au)
y_train_au = pd.DataFrame(y_train_au)
y_test_au = pd.DataFrame(y_test_au)
y_train_au.iloc[:, 0] = y_train_au.iloc[:, 0].apply(lambda x: '__label__' + str(x))
y_test_au.iloc[:, 0] = y_test_au.iloc[:, 0].apply(lambda x: '__label__' + str(x))

In [14]:
# X_train_au = new_train(X_train_au)

In [15]:
!wget -q https://raw.githubusercontent.com/VuongDS/305/main/train_au_final.csv

In [16]:
dts_au = pd.read_csv('/content/train_au_final.csv', header = None)
dts_au = dts_au.rename(columns={0: "text", 1: "spam"})

In [17]:
dts_au.shape

(380, 2)

In [18]:
dts_au.head(3)

,text,spam
0,subject ...,1
1,subject ...,1
2,subject ...,1


In [19]:
X_train_au = X_train_au.append(dts_au[['text']], ignore_index=True)

In [20]:
dts_au['spam'] = dts_au['spam'].apply(lambda x: '__label__' + str(x))

In [21]:
y_train_au = y_train_au.append(dts_au[['spam']], ignore_index=True)

In [22]:
# Create train/test dataset
X_train_au[['label']] = y_train_au
X_test_au[['label']] = y_test_au
train_dataset = X_train_au
test_dataset = X_test_au

# Convert dataset to CSV file
train_dataset[['label', 'text']].to_csv('train.txt', 
                                        index = False, 
                                        sep = ' ',
                                        header = None, 
                                        quoting = csv.QUOTE_NONE, 
                                        quotechar = "", 
                                        escapechar = " ")

test_dataset[['label', 'text']].to_csv('test.txt', 
                                        index = False, 
                                        sep = ' ',
                                        header = None, 
                                        quoting = csv.QUOTE_NONE, 
                                        quotechar = "", 
                                        escapechar = " ")

## Creating the Bag of Words model

In [23]:
!pip3 -q install fasttext
!pip3 -q install wordcloud

     |████████████████████████████████| 68 kB 4.2 MB/s 


In [24]:
# !pip3 -q install fasttext
import fasttext

**Tuning**

In [25]:
# Training the fastText classifier
model_tune = fasttext.train_supervised(
                                'train.txt', 
                                wordNgrams = 1,
                                lr = 1.3,
                                epoch = 23,
                                minn = 5, # min length of char ngrams
                                maxn = 5 # max length of char ngrams
                                )

In [26]:
# Evaluating performance on the entire test file
model_tune.test('test.txt')  

(1146, 1.0, 1.0)

In [27]:
lst_predict = []
for text in test_dataset.iloc[:, 0]:
  label, pro = model_tune.predict(text)
  lst_predict.append(list(label))
lst_predict = pd.DataFrame(lst_predict)

In [28]:
lst_predict = pd.DataFrame(lst_predict)
y_test_au['spam'] = y_test_au['spam'].replace(['__label__1'],1)
y_test_au['spam'] = y_test_au['spam'].replace(['__label__0'],0)
lst_predict[0] = lst_predict[0].replace(['__label__1'],1)
lst_predict[0] = lst_predict[0].replace(['__label__0'],0)

In [29]:
f1_score(y_test_au, lst_predict, average='binary')

1.0

In [30]:
model_tune.predict(["I give you a paper on the top of the table"]) 

([['__label__0']], [array([0.9999778], dtype=float32)])

In [31]:
def model_predict(*lst_predict):
  for lst in lst_predict:
    label, prob = model_tune.predict(lst)
    if list(label) == ['__label__0']:
      print('The sentence "{}"... is NOT a mail spam !'.format(lst[:100]))
    else:
      print('The sentence "{}"... is a mail spam !'.format(lst[:100]))
  return None

In [32]:
model_predict("I give you a paper on the top of the table")

The sentence "I give you a paper on the top of the table"... is NOT a mail spam !


In [34]:
model_predict("I have you", 'give me money now')

The sentence "I have you"... is NOT a mail spam !
The sentence "give me money now"... is a mail spam !
